In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from scipy import stats
from datetime import datetime, timedelta
from dateutil.parser import parse
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA
from IPython.display import display as dsp

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import warnings
warnings.filterwarnings('ignore')

# United States Drug Prices: Forecasting and Analysis

## Abstract

We want to explore and describe the US drug market, both in terms of general trends and the price of individual drugs on the market. The purpose of this analysis is to explore what features impact how and why drug prices change, to create a reliable metric for market volatility, and to analyse and forecast trends in drug prices. As a part of this analysis, we have explored the relationship between generic and branded drugs and their prices, net market price, percent change and date. We then used Auto Regressive and Moving Averages modeling to perform more detailed time series analysis and forecasting of both our market volatility metric and the individual drug prices.

## Data Collection

The dataset we will be performing this analysis on is the [NADAC Comparison Dataset](https://data.medicaid.gov/Drug-Pricing-and-Payment/NADAC-Comparison). The NADAC Comparison Dataset is the log of every drug price change submitted to the National Drug Acquisition Cost database. We have a record of every FDA approved drug price change to work with.

To access this data, we will be using an API to pull the dataset directly from the website. The API has some SoQL functionality implemented, we are capable of selecting the number of entires that we want to pull from the date of 11/28/2013

In [2]:
#Socrata SQL query
SODA = '$limit=500000'
URL = 'https://data.medicaid.gov/resource/444w-ftrz.csv?{}'.format(SODA)

TRAIN_SPLIT = 0.8

In [3]:
def list_unique_counts(df):
    column_name, unique_values_count, datatype = [], [], []
    results = pd.DataFrame()
    for col in df.columns:
        #print(col, '\t\t\t', len(df[str(col)].unique()))
        column_name.append(col)
        unique_values_count.append(len(df[str(col)].unique()))
        datatype.append(df[str(col)].dtype)
    
    results['Column_Name'] = column_name
    results['Unique_Value_Count'] = unique_values_count
    results['Data_Type'] = datatype
    
    dsp(results)
    print('Total Number of Changes: {}'.format(df.shape[0]))
    
    print('Total Number Observations: ', df.shape[0])

def train_split(df, TRAIN_SPLIT):
    split_point = int(len(df) * (TRAIN_SPLIT))
    train = df[:split_point]
    test = df[split_point:]
    return train, test

def strip_time(date):
    date = date.replace('T00:00:00.000', '')
    return datetime.strptime(date, '%Y-%m-%d')

def return_name(ndc_description):
    if len(ndc_description) == 0:
        return '','','',''
    drug_name, dosage, dosage_unit, drug_form = [], np.nan, np.nan, np.nan
    
    #Check and see if we are dealing with concentration
    concentration = False
    if set('%') <= set(ndc_description):
        concentration = True
        ndc_description.replace('%', '')
    
    #Create our array of words
    words = ndc_description.split()
    
    for i in range(len(words)):
        if not words[i].replace('.','').replace('%', '').isnumeric():
            drug_name.append(words[i])
            
        if words[i].replace('.','').replace('%','').isnumeric():
            dosage = float(words[i].replace('%', ''))
            
            if not concentration:
                try:
                    dosage_unit = words[i + 1]
                    drug_form = ' '.join(words[i + 2:])
                except:
                    drug_form = ' '.join(words[i:])
                    #print(ndc_description, concentration)
                break
            
            if concentration:
                dosage_unit = 'concentration'
                drug_form = ' '.join(words[i + 1:])
                break
            
    drug_name = ' '.join(drug_name)
    return pd.Series([drug_name, dosage, dosage_unit, drug_form])

def drop_na_printout(dataframe):
    i = dataframe.shape[0]
    j = dataframe.dropna().shape[0]
    
    percent_loss = (i - j) / i * 100
    
    print(percent_loss, '% data lost in cleaning.')
    
    return dataframe.dropna()

def ts_lin_test(series):
     return stats.linregress(range(len(series)), series.values)

### Data Description

The data herein contains price changes applied to the database. Each observation contains an 'ndc_description' string, which encapsulate the drug name, the dosage (including units of measurement) and the drug format. The 'ndc' column contains a serial particular to the label, product and package. The 'old_nadac_per_unit' and 'new_nadac_per_unit' describe the USD price listed per unit. 

The 'classification_for_rate_setting' tells us if the drug whose rate is being changed is labeled as generic or branded. The 'percent_change' gives us the percent change from the old to new price. The 'primary_reason' gives us the reason listed for the change and the 'start_date' tells us when the change is applied to the database.

### Data Cleaning and Preparation

Our most intensive issue at this point is the extraction of the drug name and dosage information from the ndc description. To perform a more comprehensive analysis, we will need to scale the price to the dosage of the drug. Not every ndc description follows the same format of drug name + dosage measure + units of measurement + format. A few drugs do not even include dosage measures and units of measurements, so we can expect some data loss at this point. We are able to extract information from drugs sold as a solution and in a solid form.

In [4]:
raw = pd.read_csv(URL, parse_dates = True)

#Convert dates to date format
raw['start_date'] = raw['start_date'].apply(lambda x: strip_time(x))
raw['end_date'] = raw['end_date'].apply(lambda x: strip_time(x))

#Remove Effective Date
raw = raw.drop('effective_date', 1)

In [ ]:
print(raw.columns)

#Create 'days_effective' and 'day_count' columns
raw['days_effective'] = (raw['end_date'] - raw['start_date'])
raw['days_effective'] = raw['days_effective'].apply(lambda x: x.days)

raw['days_count'] = (raw['start_date'] - min(raw['start_date']))
raw['days_count'] = raw['days_count'].apply(lambda x: x.days)

#Create price change column
raw['price_change'] = raw['new_nadac_per_unit'] - raw['old_nadac_per_unit']

#Create 'month-year' column because most changes happen in the middle of every month
raw['year-month'] = raw['start_date'].apply(lambda x: x.strftime('%Y-%m'))

#Add drug name column, we want to ignore dosage and format
raw[['drug_name', 'dosage','dosage_unit','drug_form']] = \
raw['ndc_description'].apply(lambda x: return_name(x))

#Remove all first-date entries
if len(raw['start_date'].unique()) > 1:
    raw.drop(raw[raw['start_date'] == min(raw['start_date'])].index, inplace = True)

#Remove last month of data, this is typically truncated by the limitations of our API call
if len(raw['year-month']) > 3:
    raw.drop(raw[raw['year-month'] == max(raw['year-month'])].index, inplace = True)

    
#Generate change_count df
change_count = raw['ndc'].groupby('start_date').count()
print('Our data represents the range of dates between {} and {}'.format(change_count.index.min().strftime('%Y-%m-%d'),
                                                                        change_count.index.max().strftime('%Y-%m-%d')))

Index(['ndc_description', 'ndc', 'old_nadac_per_unit', 'new_nadac_per_unit',
       'classification_for_rate_setting', 'percent_change', 'primary_reason',
       'start_date', 'end_date', 'days_effective', 'days_count',
       'price_change', 'year-month', 'drug_name', 'dosage', 'dosage_unit',
       'drug_form'],
      dtype='object')


In [ ]:
#Remove observations with missing entries.
raw = drop_na_printout(raw)

In [ ]:
#Scale price to dosage
raw['daily_max_dosage'] = raw.groupby(['drug_name', 'start_date', 'dosage_unit'])['dosage'].transform('max')
raw['scaled_old_per_unit'] = raw['daily_max_dosage'] / raw['dosage'] * raw['old_nadac_per_unit']
raw['scaled_new_per_unit'] = raw['daily_max_dosage'] / raw['dosage'] * raw['new_nadac_per_unit']
raw['scaled_percent_change'] = (raw['scaled_new_per_unit'] - raw['scaled_old_per_unit']) \
/ raw['scaled_old_per_unit'] * 100

In [9]:
change_count.head()

,ndc_description,ndc,old_nadac_per_unit,new_nadac_per_unit,classification_for_rate_setting,percent_change,primary_reason,end_date,days_effective,days_count,price_change,year-month,drug_name,dosage,dosage_unit,drug_form
start_date,,,,,,,,,,,,,,,,
2013-12-18,62,62,62,62,62,62,62,62,62,62,62,62,62,61,61,61
2013-12-25,12,12,12,12,12,12,12,12,12,12,12,12,12,11,11,11
2014-01-08,408,408,408,408,408,408,408,408,408,408,408,408,408,318,318,318
2014-01-15,18543,18543,18543,18543,18543,18543,18543,18543,18543,18543,18543,18543,18543,16024,16024,16024
2014-01-22,19,19,19,19,19,19,19,19,19,19,19,19,19,15,15,15


In [ ]:
#Create drugs dataframe
drugs = raw[['drug_name','old_nadac_per_unit','new_nadac_per_unit']].groupby(['drug_name']).mean().rename(
    {'old_nadac_per_unit' : 'old_average_per_unit','new_nadac_per_unit': 'new_average_per_unit'}, axis = 1)
drugs['percent_average_change'] = (drugs['new_average_per_unit'] 
                                   - drugs['old_average_per_unit']) / drugs['new_average_per_unit'] * 100

In [ ]:
max_price, min_price, max_dosage = [], [], []
for drug in drugs.index:
    max_price.append(max(raw[raw['drug_name'] == drug]['new_nadac_per_unit']))
    min_price.append(min(raw[raw['drug_name'] == drug]['new_nadac_per_unit']))
    max_dosage.append(max(raw[raw['drug_name'] == drug]['dosage']))
drugs['max_per_unit'] = max_price
drugs['min_per_unit'] = min_price
drugs['max_dosage'] = max_dosage
drugs.head()

In [ ]:
plt.title('Max Price per Unit Distribution')
plt.hist(drugs['max_per_unit'])

plt.xlabel('Price per Unit')
plt.show()

## Exploratory Data Analysis

### Metadata

The first order of business with our data is to discern what datatype and how many unique entries are present in our raw dataset.

In [ ]:
print('\nUnique Values in Raw Data per Column:')
list_unique_counts(raw)

Of interest are two observations:

First, the days effective, which describes how many days are between the 'start date' and the 'end date'. There are only two unique values listed in this column, 6 and 7. As the database updates on a weekly basis, this makes sense. We will be using the start date for our time series analysis.

Second, there are an overwhelming numnber of ndc values as there are drug names, or even ndc descriptions. The ndc code represents specific batches and labels, which is information we are unable to take into account. It is far more useful to group by drug name or ndc description.

In [ ]:
print('Top value counts for NDC:')
dsp(raw['ndc'].value_counts().sort_values(ascending = False).head())

print('\nTop value counts for NDC description:')
dsp(raw['ndc_description'].value_counts().sort_values(ascending = False).head())

print('\nTop value counts for drug name:')
dsp(raw['drug_name'].value_counts().sort_values(ascending = False).head())

Here we can see the number of entries per ndc code and ndc description. The specificity of the ndc code means for the intent of our analysis, this can be ignored. The ndc description list also has an issue. Depeding on the number of unique dosages, drug names will be repeated multiple times. Since we are interested in individual drugs, we must consolidate this information into the drug name.

### Data Distribution2350

We create two dataframes, the first which contain columns from the raw dataset we are interested in for further analysis. The second with all numeric data remove. As we may also be interested in exploring outliers later.

In [ ]:
data = raw[['drug_name','start_date', 'days_count', 'year-month', 'old_nadac_per_unit', 'new_nadac_per_unit', \
            'price_change','scaled_old_per_unit', 'scaled_new_per_unit', 'percent_change', 'scaled_percent_change']]

#numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data_out_rm = data[(np.abs(stats.zscore(data[['percent_change']])) < 3).all(axis = 1)]

#### Percent Change

Percent change is a good metric for understanding how volatile individual drugs are, as it is a ratio measurement of change. However, it is not a good indicater to use when comparing drugs to other drugs as there is no indication of magnitude of price. For example, if a drug were to increase from 0.01$ to 0.02$, we would see a notable 50% increase, within the drug history, but the price itself would not be noteworthy when comparing a drug that changed from 100$ to 200$ per unit.

In [ ]:
plt.title('Percent Change Distribution')
plt.hist(data['percent_change'])
plt.xlabel('% change')
plt.ylabel('N Obs')
plt.show()

plt.title('Percent Change Distribution, Outliers Removed')
plt.hist(data_out_rm['percent_change'])
plt.xlabel('% change')
plt.ylabel('N Obs')
plt.show()

plt.title('Percent Change per Date')
plt.scatter(data['start_date'], data['percent_change'])
plt.xticks(rotation = 70)
plt.xlabel('Date')
plt.ylabel('Percent Change')

plt.show()

In [ ]:
raw.groupby(['drug_name']).max().sort_values('percent_change', ascending = False)['percent_change'][:10]

#### Price per Unit: de facto and scaled

On the surface, even with outliers removed, it is clear that the distribution of prices per unit over the entire log is a steep exponential distribution. This shows that there are lots of drug changes where the final price of the drug change is fairly small. Our suspicions are confirmed when we group our data by average price per drug name. We are using the scaled price when observing this distribution as having multiple denominations of drugs may introduce scaling issues that prevent our data from fully reflecting the reality of the data.

In [ ]:
plt.title('New Price Per Unit')
plt.hist(data_out_rm['new_nadac_per_unit'], color = 'green')
plt.xlabel('Price per Unit')
plt.ylabel('N Obs')
plt.show()

plt.title('New Price Per Unit (0, 1)')
plt.hist(data['new_nadac_per_unit'], range = (0, 1), color = 'green')
plt.xlabel('Price per Unit')
plt.ylabel('N Obs')
plt.show()

drugmean = data.groupby(['drug_name']).mean()[['new_nadac_per_unit', 'scaled_new_per_unit']]
plt.title('Average Price per Drug Name Distribution')
plt.hist(drugmean['scaled_new_per_unit'], color = 'purple')
plt.xlabel('Price')
plt.ylabel('N Drugs')
plt.show()

plt.title('New Price to Date')
plt.scatter(data['start_date'], data['new_nadac_per_unit'], color = 'green')
plt.xticks(rotation = 70)
plt.xlabel('Date')
plt.ylabel('Price per Unit')
plt.show()

#Price Differential
#Remove Outliers

There appears to be some form of seasonality in the scatterplot for the price per unit data which does not appear to be reflected in the percent change scatterplot. This implies that the outliers here are indicitive of a single drug whose value is changed on a monthly or near monthly basis.

When we compare the new nadac per unit to the scaled new nadac per unit, we can see that we have slightly altered the distribution. Our distribution has become marginally less sharp. This is a direct result of how we performed the scaling, where we scaled the drug price to the highest dosage, multiplying lower dosages by a scaling factor to raise the price to reflect the amount of active ingredient included.

In [ ]:
plt.title('Scaled New Price Per Unit (0, 1)')
plt.hist(data['new_nadac_per_unit'], range = (0, 1), color = 'green', alpha = 0.5)
plt.hist(data['scaled_new_per_unit'], range = (0, 1), color = 'red', alpha = 0.5)
red_patch = mpatches.Patch(color='red', label='Scaled Price per Unit')
green_patch = mpatches.Patch(color='green', label='Price per Unit')
plt.legend(handles = [red_patch, green_patch])
plt.xlabel('Price per Unit')
plt.ylabel('N Obs')
plt.show()

plt.title('Scaled New Price to Date')
plt.scatter(data['start_date'], data['scaled_new_per_unit'], color = 'red')
plt.xticks(rotation = 70)
plt.xlabel('Date')
plt.ylabel('Price per Unit')
plt.show()

#Overlap Scaled and raw
#Don't hard code the range

#### Price Change

In [ ]:
plt.title('Price Change Distribution')
plt.hist(data_out_rm['price_change'])
plt.xlabel('price change')
plt.ylabel('N Obs')
plt.show()

plt.title('Price Change Distribution (-3, 3)')
plt.hist(data_out_rm['price_change'], range = (-3, 3))
plt.xlabel('price change')
plt.ylabel('N Obs')
plt.show()

plt.title('Price Change per Date')
plt.scatter(data['start_date'], data['price_change'])
plt.xticks(rotation = 70)
plt.xlabel('Date')
plt.ylabel('price change')

plt.show()

The price change reflects the price differential between old and new price. This metric is more useful for comparing magnitude of changes between drugs rather than performance within drugs themselves. We may further improve the comparison by introducing a percentage factor. A high percentage with a high price differential implies prominence to the change as this implies a cheap drug becoming more expensive.

In [ ]:
print('\nTop drug name max price changes:')
dsp(raw.groupby(['drug_name']).max().sort_values('price_change', ascending = False)['price_change'].head())

#### Correlation Testing

In [ ]:
sns.heatmap(data[['days_count', 'scaled_new_per_unit', 'percent_change', 'price_change']].corr())
plt.title('Raw Data Correlation Heatmap')
plt.show()

Here we have our correlation matrix between some of our features of interest. Some points of interest here:

1. We can infer that the number of days since the start and the percent change are strongly uncorrelated between individual observations. This implies that the percent change does not notably change with time.

2. We can see that the price change and the scaled new price per unit are fairly well correlated, whereas the percent change is not. From this we can infer that the price of the drug is not a factor in the percent change of the drug.

Naturally, this correlation map is a good indicator of correlating features within individual observations, but terrible at describing general trends within the data. There is the issue of sampling, the observations submitted on a day with fewer observations will be more heavily weighted than on a day with many observations when it comes to correlations with time.

In [ ]:
sns.heatmap(data.groupby('year-month').mean()\
            [['days_count','scaled_new_per_unit', 'percent_change', 'price_change']].corr())

plt.title('Daily Averages Correlation Heatmap')
plt.show()

Price = A * e ^ f(percent)

df / dt < 0
dPrice / dt > 0

Here, we have grouped our data by month, and we can already see a greater spectrum of correlation. Again, we can note that the percent change is negatively correlated with the day count, but the price change is. This, combined with the correlation between day count and price per unit, is indicative of slowing exponential growth in the drug prices as every month drug prices increase by some diminishing percentage.

#### Generic VS Branded

In [ ]:
print('{}% of changes are to generic medicines.'.format(round(len(raw[raw['classification_for_rate_setting'] == 'G'])
                                                              / raw.shape[0], 3) * 100))
raw_generic = raw[raw['classification_for_rate_setting'] == 'G']['percent_change']
raw_branded = raw[raw['classification_for_rate_setting'] == 'B']['percent_change']

#Compare how often and the magnitude of change between generic and branded medicine.
plt.title('Generic and Branded Percent Change Distribution')
plt.hist(raw_generic, normed = True)
plt.hist(raw_branded, normed = True)

plt.xlabel('Percent Change')
plt.ylabel('Proportions')
plt.show()


plt.title('Generic and Branded Percent Change Distribution (Outliers Excluded)')
plt.hist(raw_generic[(np.abs(stats.zscore(raw_generic)) < 3)], normed = True)
plt.hist(raw_branded[(np.abs(stats.zscore(raw_branded)) < 3)], normed = True)

plt.xlabel('Percent Change')
plt.ylabel('Proportions')
plt.show()

#Print Stats
print('Mean Percent Change for Generic Drugs: {} percentage points'.format(round(raw_generic.mean() ,2)))
print('Mean Percent Change for Branded Drugs: {} percentage points'.format(round(raw_branded.mean() ,2)))

print('Standad Deviation for Generic Drugs: {} percentage points'.format(round(raw_generic.std() ,2)))
print('Standad Deviation for Branded Drugs: {} percentage points'.format(round(raw_branded.std() ,2)))

#T-test
t_stat, p_val = stats.ttest_ind(raw_generic, raw_branded)

print('T-Statistic: {}'.format(round(t_stat)))
print('P-Value: {}'.format(round(p_val)))


### Metrics for General Volatility

What do we mean by volatility?

Volatility is a measure of a system's proclivity to change. We want to create a metric that best describes the volatility of drug prices in the US. The changelog format of our dataset lends it well to exploring various measures of volatility.

#### Changes per Day

In [ ]:
plt.title('Changes Implemented Per Day')
plt.plot(change_count.index.strftime('%Y-%m-%d'), change_count.values, '.')
plt.xticks(np.arange(0,len(change_count.index), round((len(change_count.index)) / 8)), rotation = 70)
plt.xlabel('Date')
plt.ylabel('N Changes')
plt.show()

There are two glaring issues to note with using the total number of changes per day as a metric for market volatility. First, our database updates once a week, so the highest resolution we can get from the data is by week. Second, the vast majority of changes are submitted every four weeks. There may be something of interest in comparing the metrics of changes submitted on low volume days vs high volume days, but this is beyond the scope of this analysis. For now, we will look at the data at a montly resolution.

#### Changes per Month

On the surface, changelog entry counts make sense as a metric for market volatility. The more changes that are submitted, the more the market changes. This encapsulates new drugs entering the market and major drug pricing policy changes.

In [ ]:
mnth_count = raw['year-month'].value_counts().sort_index()
plt.title('Changes per Month')
plt.plot(mnth_count, '.')
plt.xticks(np.arange(0,len(mnth_count.index), round((len(mnth_count.index)) / 8)), rotation = 70)
plt.xlabel('Date')
plt.ylabel('N Changes')
plt.show()

#Linearity test
#slope, intercept, r_value, p_value, std_err = 
stats.linregress(range(len(mnth_count)), mnth_count.values)

We can clearly see a linear trend within the data here, with a fairly high r-value. This data would be good to use in a model as a result of its linearity. Conceptually, using gross number of changes per period also makes sense, as a period with a higher number of changes could be considered more 'volatile'.

#### Average Percent Change per Month

We can also measure volatility by looking at the average percent change per month. This gives us an indication as to how much each drug is changing in comparison to itself. This is a good metric for taking into account the magnitude rate of change.

In [ ]:
mnth_avgs = raw.groupby(['year-month']).mean()
plt.title('Average Monthly Percent Change')
plt.plot(mnth_avgs['percent_change'], '.')
plt.xticks(np.arange(0,len(mnth_avgs.index), round((len(mnth_avgs.index)) / 8)), rotation = 70)
plt.xlabel('Date')
plt.ylabel('Average Price Change')
plt.show()

stats.linregress(range(len(mnth_avgs['percent_change'])), mnth_avgs['percent_change'].values)

Our linear regression test shows the negative correlation we had observed before, but the linearity is fairly poor.

In [ ]:
plt.title('Average Monthly Percent Change Distribution')
mnth_avgs['percent_change'].hist()
plt.xlabel('Percent Change')
plt.ylabel('Observations')

#### Total Price Gain/Loss per Month

Another potential metric for market volatility we may use is the total price gain/loss per month. Essentially, we take the sum of every entry in the changelog per month as our metric. This is a direct measure of price, but is more sensitive to individual drugs with many different dosages being changed.

In [ ]:
raw['total_price_change'] = raw['new_nadac_per_unit'] - raw['old_nadac_per_unit']
mnth_sum = raw.groupby(['year-month']).sum()

In [ ]:
plt.title('Total Price Change per Month')
plt.plot(mnth_sum['total_price_change'], '*')
plt.xticks(np.arange(0,len(mnth_sum.index), round((len(mnth_sum.index)) / 8)), rotation = 70)
plt.xlabel('Date')
plt.ylabel('Total Price Change')
plt.show()
stats.linregress(range(len(mnth_sum['total_price_change'])), mnth_sum['total_price_change'].values)

We can see positive trends in the data, however, the linearity is still fairly low.

Unfortunately, limited by our dataset, we do not have a complete picture of the supply and demand for drugs in the US. For that, we would need to introduce data which included information regarding how many units of every drug are purchased. However, we can infer some information regarding the volatility with any of the data above. To that end I will be using the total count log as my metric of volatility.

### Preliminary ARIMA modeling

Auto Regression and Moving Averages is an ideal model to use with this data. ARIMA modeling is exceptional at working with time series data as it is exceptional at picking up longer term linear trends (AR) and short term/seasonal events (MA). Lets look at how well ARIMA modeling works with our data.

In [ ]:
#Calculate the correlation within these and their respective p-values
count_arima = ARIMA(mnth_count, order=(0,1,1)) #Play around with the order (110 101)
p_chng_arima = ARIMA(mnth_avgs['percent_change'], order=(1,1,1))
tot_chng = ARIMA(mnth_sum['total_price_change'], order=(1,1,1))

count_arima_fit = count_arima.fit()
p_chng_arima_fit = p_chng_arima.fit()
tot_chng_fit = tot_chng.fit()

print(count_arima_fit.summary())
print('\n\n\n')
print(p_chng_arima_fit.summary())
print('\n\n\n')
print(tot_chng_fit.summary())

In [ ]:

count_arima_fit.plot_predict(start = mnth_count.index[1],end = max(mnth_count.index))
plt.title('Entry Count per Month ARIMA Model')
plt.ylabel('Entries')

p_chng_arima_fit.plot_predict(start = mnth_avgs.index[1],end = max(mnth_avgs.index))
plt.title('Average Percent Change per Month ARIMA Model')
plt.ylabel('Percent Change')

tot_chng_fit.plot_predict(start = mnth_sum.index[1],end = max(mnth_sum.index))
plt.title('Total Price Change per Month ARIMA Model')
plt.ylabel('Total Price Change per Day')
plt.show()

Unfortunately, our chosen metric for 

### Model Performance

We can evaluate model performance by observing the plot and histogram of the residuals, that is the difference at every point between observed and predicted values for our measure at every available point in time. Ideally, the residuals scatter plot would show no correlation between day and magnitude, and that the histogram for the residuals would show a normal distribution

In [ ]:
residuals_count = pd.DataFrame(count_arima_fit.resid)
residuals_count.plot(legend=False)
plt.title('Time Series of Count Residuals')
residuals_count.hist(bins=20)
plt.title('Histogram of Count Residuals')

residuals_p_chng = pd.DataFrame(p_chng_arima_fit.resid)
residuals_p_chng.plot(legend=False)
plt.title('Time Series of Mean Percent Change Residuals')
residuals_p_chng.hist(bins=20)
plt.title('Histogram of Mean Percent Change Residuals')

residuals_tot_chng = pd.DataFrame(tot_chng_fit.resid)
residuals_tot_chng.plot(legend=False)
plt.title('Time Series of Total Change Residuals')
residuals_tot_chng.hist(bins=20)
plt.title('Histogram of Total Change Residuals')

Unfortunately, in terms of monthly data, we do not have very many datapoints to fill out our histogram. We can still use these models to make decent predictions with the resolution we have.

It appears that the ARIMA model when applied to the total monthly entry counts performs best. The distribution is begining to look normal and the residuals do not appear to have much of a correlation. We may increase the number of datapoints at a later time; however, the best approach to doing so would be to use some form of distributed computing methodology given the sheer size of the entire database.

#### Drug Name: Record Count and Price Change

Before we move on to forecasting, lets look at the correlation between features specific to trends within individual drugs. We want to know how the record counts, the average percent change and the price correlate with one another between individual drugs. This may offer us some clues as to what features would best be used to predict the prices of our drugs.

In [ ]:
#Create a dataframe for every drug name
drugs = pd.DataFrame(index = raw['drug_name'].unique())

drugs['value_counts'] = raw.groupby(['drug_name']).count()['ndc']
drugs['average_price_per_unit'] = raw.groupby(['drug_name']).mean()['new_nadac_per_unit']
drugs['average_scaled_price_per_unit'] = raw.groupby(['drug_name']).mean()['scaled_new_per_unit']
drugs['average_percent_change'] = raw.groupby(['drug_name']).mean()['percent_change']

drugs.head()

In [ ]:
plt.title('Pearson Correlation')
sns.heatmap(drugs[['value_counts','average_scaled_price_per_unit', 'average_percent_change']].corr())

Unfortunately, there is not much intercorrelation between any of our features of interest. The average percent change is very slightly correlated with the price per unit when we group by drugs, implying that more expensive drugs tend to receive higher percent increases. The average percent change appears to be nearly perfectly uncorrelated with the value counts, this implies that the value counts have nothing to do with what percent changes are applied to the drug prices.

### Forecasting

#### Individual Drugs

__By NDC Description__

In [ ]:
#Find most frequently changed ndc prices
top_n_drug_name = raw['drug_name'].value_counts().sort_values(ascending = False)[:25]

### Forecasting

#### Market Volatility

In [ ]:
#Now we will forecast the next two years of monthly entry counts
#We have already trained the model on the training set, and now we can validate it on the test set.

#count_arima_fit.predict()
size = int(len(mnth_count) * TRAIN_SPLIT)
test =  mnth_count[size:len(mnth_count)]

mnth_count.sort_index()
#Create prediction array


#Create array of timestamps:

#plt.plot(mnth_count, color = 'blue')

count_arima_fit.plot_predict(start = mnth_count.index[1],end = max(mnth_count.index))
plt.title('Entry Counts per Date Forecast')
#plt.xticks(np.arange(0,len(mnth_count.index), round((len(mnth_count.index)) / 8)), rotation = 70)
plt.legend(loc = 'upper left')
plt.xlabel('Date')
plt.ylabel('N Changes')
plt.show()

In [ ]:
my_date = mnth_count.index[2]
print(my_date, type(my_date))

print(min(mnth_count.index))
print(mnth_count.index)
print(mnth_count.shape)
print(type(min(mnth_count.index)))
#print(type(datetime.timestamp(min(mnth_count.index))))

print(min(mnth_count.index), type(min(mnth_count.index)))

#count_arima_fit.plot_predict(start = mnth_count.index[1],end = mnth_count.index[-1])

print(mnth_count.index[-1])
print(type(mnth_count.index[-1]))

#### Individual Drug Names

Rather than attempting to create and explore a model for every one of the 2350 unique drug names available in the dataset, we will instead be looking at the the top 25 most often changed drugs. The sampling methodology we use here is biased; however, we may be able to do some inferrence between this and the trends in drug prices we have seen before.

In [ ]:
#Create dictionary to store models.
arima_models = {}
coefficient = []
p_value = []

#LISINOPRIL

for i, drug in enumerate(top_n_drug_name.index):
    drug_history = raw[raw['drug_name'] == str(drug)].groupby(['year-month']).mean()
    try:
        arima_models[drug] = ARIMA(drug_history['scaled_new_per_unit'], order = (1,1,1)).fit()
    #Fall through for non-stationarity
    except:
        arima_models[drug] = ARIMA(drug_history['scaled_new_per_unit'], order = (0,1,1)).fit()
    
    #Plot results
    arima_models[drug].plot_predict(start = drug_history.index[1],end = max(drug_history.index))
    plt.title('{} history'.format(drug))
    plt.show()

An interesting trend we can see amongst the top 25 highest change count drugs is that most of these have a general downward trend. I would hypothesize that this is a result of such drugs being sold under generic labels, being sold with more dosages and more manufacturers having manufacturing rights for these drugs, thus increasing the number of ndc labels that refer to these particular drug names.

For example, ibuprofen is a common pain relief and anti-inflamitory over the counter drug widely available in almost every grocery store. There are many different generic label brands that manufacture ibuprofen. Exploring the relationship between availability, usage, and demand for drugs would be an interesting and fruitful extension to this analysis, provided such data could be found.

In [ ]:
#Create nice visual aid
#for i, drug in enumerate(top_n_drug_name.index):
#    plt.subplot(5, 5, i + 1)
#    arima_models[drug].plot_predict(start = drug_history.index[1],end = max(drug_history.index))
    
#plt.show()

print(raw[raw['start_date'] == raw['start_date'][0]]['percent_change'].mean())

In [ ]:
#Check the average percent change between drug at beginning and end of time frame

## Conclusion

We were able to pull out many interesting insights within the drug pricing market.

At its most general level, we can see that drug prices are undergoing exponential growth. 
We were able to explore a metric for market volatility and create a servicable model for forecasting purposes. As well as create a dictionary of models for the top 25 most common drug name entries.

__NOTE__: Expand conclusion, summary.


## Further Work

I would have liked to encorporate a measure of the effects of major medical administration policy changes using A/B testing. With the current discourse surrounding the US healthcare system, it would be interesting to see what policy changes have had what effects on drug prices. We could get a more comprehensive picture of healthcare changes by encorporating other datasets from other sections of the industry, such as hospital billing and procedures. As well as encorporating data from other countries for comparison.

As mentioned before, when looking at a purly economic standpoint, we could get a more comprehensive picture of drug performances by looking at some metrics for drug supply and demand. What we have been using only has given us a single facet of the marketplace.

Finally, if we could find a comprehensive database which lists all the use cases and the effects of the drugs we have gone over, we could perform some insightful clustering that could tell us how similar and different drugs perform on the marketplace.